In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

In [0]:
TRAIN_DATADIR = "/content/drive/My Drive/CNN/image/train"
TEST_DATADIR = "/content/drive/My Drive/CNN/image/test"
CATEGORIES = ["DRAM 1","DRAM 2","DRAM 3"]

In [0]:
def create_training_data(IMG_SIZE):
  training_data = []
  test_data = []
  X_train = []
  y_train = []
  X_test =  []
  y_test =  []

  for category in CATEGORIES:
    path = os.path.join(TRAIN_DATADIR,category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      new_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      img_array = cv2.resize(new_array,(IMG_SIZE,IMG_SIZE))
      training_data.append([img_array,class_num])

  
  for category in CATEGORIES:
    path = os.path.join(TEST_DATADIR,category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      new_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      img_array = cv2.resize(new_array,(IMG_SIZE,IMG_SIZE))
      test_data.append([img_array,class_num])

  random.shuffle(training_data)
  random.shuffle(test_data)

  for sample,label in training_data:
    X_train.append(sample)
    y_train.append(label)
  X_train = np.array(X_train).reshape(-1,IMG_SIZE,IMG_SIZE,1)
  X_train = X_train / 255.0

  for sample,label in test_data:
    X_test.append(sample)
    y_test.append(label)
  X_test = np.array(X_test).reshape(-1,IMG_SIZE,IMG_SIZE,1)
  X_test = X_test / 255.0
  return [X_train,y_train,X_test,y_test]

In [0]:
def cnn_model(X):
  model = Sequential()
  model.add(Conv2D(3,(3,3),strides= (1,1),input_shape = X.shape[1:]))
  model.add(Activation("relu"))

  model.add(Dropout(0.2))
  model.add(Conv2D(64,(3,3),strides= (1,1)))
  model.add(Activation("relu"))

  model.add(MaxPooling2D(pool_size=(2,2),strides = (2,2)))

  model.add(Dropout(0.2))
  model.add(Conv2D(128,(3,3),strides= (1,1)))
  model.add(Activation("relu"))

  model.add(MaxPooling2D(pool_size=(2,2),strides = (2,2)))

  model.add(Conv2D(192,(3,3),strides= (1,1)))
  model.add(Activation("relu"))

  model.add(MaxPooling2D(pool_size=(2,2),strides = (2,2)))
  
  model.add(Flatten())

  model.add(Dense(2048,activation="relu"))
  model.add(Dense(3,activation="softmax"))
  model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
  return model

In [0]:
img_size = 64
X_train,y_train,X_test,y_test = create_training_data(img_size)

In [0]:
train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                             zca_whitening = True, channel_shift_range=0.1)

train_datagen.fit(X_train)

test_datagen = ImageDataGenerator(width_shift_range = 0.1,
                             zca_whitening = True, channel_shift_range=0.1)
test_datagen.fit(X_test)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
train_generator = train_datagen.flow(X_train,y_train,batch_size = X_train.shape[0],shuffle=True)
test_generator = test_datagen.flow(X_test,y_test,batch_size = X_train.shape[0],shuffle = True)

In [0]:
augmented_model = cnn_model(X_train)
training_details = augmented_model.fit_generator(train_generator,
                              steps_per_epoch = 5,
                              epochs = 1000,
                              validation_data = test_generator,
                              validation_steps =5)

In [0]:
import matplotlib.pyplot as plt
plt.plot(training_details.history['val_acc'])
plt.plot(training_details.history['acc'])
plt.show()

In [0]:
import pickle
with open('/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(training_details.history, file_pi)

In [0]:
history = pickle.load(open('/trainHistoryDict', "rb"))